<a href="https://colab.research.google.com/github/shreyasat27/pennylane-27524/blob/main/He2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here I'll try to simulate He2 and find the ground state energy of the molecule

In [ ]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 42.9 MB/s eta 0:00:00


In [8]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
import time

symbols = ["He", "He"]
geometry = np.array([[0.0, 0.0, -1.5], [0.0, 0.0, 1.5]])
molecule = qchem.Molecule(symbols, geometry)

In [9]:
# H, qubits = qchem.molecular_hamiltonian(
#     molecule,
#     active_electrons=2,
#     active_orbitals=4
# )
dataset = qml.data.load('qchem', molname="He2")[0]
H, qubits = dataset.hamiltonian, len(dataset.hamiltonian.wires)


active_electrons = 4

singles, doubles = qchem.excitations(active_electrons, qubits)

print(f"Total number of excitations = {len(singles) + len(doubles)}")

Total number of excitations = 26


Note that we have a total of 24 excitations which can be represented by the same number of excitation gates 4. Let’s now use an AdaptiveOptimizer implemented in PennyLane to construct an adaptive circuit.



In [10]:
singles_excitations = [qml.SingleExcitation(0.0, x) for x in singles]
doubles_excitations = [qml.DoubleExcitation(0.0, x) for x in doubles]
operator_pool = doubles_excitations + singles_excitations

In [11]:
hf_state = qchem.hf_state(active_electrons, qubits)
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def circuit():
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
    return qml.expval(H)

In [12]:
opt = qml.optimize.AdaptiveOptimizer()
for i in range(len(operator_pool)):
    circuit, energy, gradient = opt.step_and_cost(circuit, operator_pool)
    if i % 3 == 0:
        print("n = {:},  E = {:.8f} H, Largest Gradient = {:.3f}".format(i, energy, gradient))
        print(qml.draw(circuit, decimals=None)())
        print()
    if gradient < 3e-3:
        break

n = 0,  E = -5.71032085 H, Largest Gradient = 0.114
0: ──X─────┤ ╭<𝓗>
1: ──X─────┤ ├<𝓗>
2: ──X─╭G²─┤ ├<𝓗>
3: ──X─├G²─┤ ├<𝓗>
4: ────├G²─┤ ├<𝓗>
5: ────╰G²─┤ ├<𝓗>
6: ────────┤ ├<𝓗>
7: ────────┤ ╰<𝓗>

n = 3,  E = -5.72157664 H, Largest Gradient = 0.137
0: ──X─────╭G²─╭G²─────┤ ╭<𝓗>
1: ──X─────│───├G²─╭G²─┤ ├<𝓗>
2: ──X─╭G²─│───│───├G²─┤ ├<𝓗>
3: ──X─├G²─├G²─│───│───┤ ├<𝓗>
4: ────├G²─├G²─│───│───┤ ├<𝓗>
5: ────╰G²─│───│───├G²─┤ ├<𝓗>
6: ────────│───├G²─╰G²─┤ ├<𝓗>
7: ────────╰G²─╰G²─────┤ ╰<𝓗>

n = 6,  E = -5.73249869 H, Largest Gradient = 0.118
0: ──X─────╭G²─╭G²─────╭G²─────────┤ ╭<𝓗>
1: ──X─────│───├G²─╭G²─├G²─╭G²─────┤ ├<𝓗>
2: ──X─╭G²─│───│───├G²─│───├G²─╭G²─┤ ├<𝓗>
3: ──X─├G²─├G²─│───│───│───│───├G²─┤ ├<𝓗>
4: ────├G²─├G²─│───│───├G²─├G²─│───┤ ├<𝓗>
5: ────╰G²─│───│───├G²─╰G²─│───│───┤ ├<𝓗>
6: ────────│───├G²─╰G²─────│───├G²─┤ ├<𝓗>
7: ────────╰G²─╰G²─────────╰G²─╰G²─┤ ╰<𝓗>

n = 9,  E = -5.74011171 H, Largest Gradient = 0.015
0: ──X─────╭G²─╭G²─────╭G²─────────╭G²─╭G²─────┤ ╭<𝓗>
1: ──X─────│───

The resulting energy matches the exact energy of the ground electronic state of LiH, which is -7.8825378193 Ha, within chemical accuracy. Note that some of the gates appear more than once in the circuit. By default, AdaptiveOptimizer does not eliminate the selected gates from the pool. We can set drain_pool=True to prevent repetition of the gates by removing the selected gate from the operator pool.

In [13]:
@qml.qnode(dev)
def circuit():
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
    return qml.expval(H)

opt = qml.optimize.AdaptiveOptimizer()
for i in range(len(operator_pool)):
    circuit, energy, gradient = opt.step_and_cost(circuit, operator_pool, drain_pool=True)
    if i % 2 == 0:
        print("n = {:},  E = {:.8f} H, Largest Gradient = {:.3f}".format(i, energy, gradient))
        print(qml.draw(circuit, decimals=None)())
        print()
    if gradient < 3e-3:
        break

n = 0,  E = -5.71032085 H, Largest Gradient = 0.114
0: ──X─────┤ ╭<𝓗>
1: ──X─────┤ ├<𝓗>
2: ──X─╭G²─┤ ├<𝓗>
3: ──X─├G²─┤ ├<𝓗>
4: ────├G²─┤ ├<𝓗>
5: ────╰G²─┤ ├<𝓗>
6: ────────┤ ├<𝓗>
7: ────────┤ ╰<𝓗>

n = 2,  E = -5.71755852 H, Largest Gradient = 0.126
0: ──X─────╭G²─╭G²─┤ ╭<𝓗>
1: ──X─────│───├G²─┤ ├<𝓗>
2: ──X─╭G²─│───│───┤ ├<𝓗>
3: ──X─├G²─├G²─│───┤ ├<𝓗>
4: ────├G²─├G²─│───┤ ├<𝓗>
5: ────╰G²─│───│───┤ ├<𝓗>
6: ────────│───├G²─┤ ├<𝓗>
7: ────────╰G²─╰G²─┤ ╰<𝓗>

n = 4,  E = -5.72632182 H, Largest Gradient = 0.107
0: ──X─────╭G²─╭G²─────╭G²─┤ ╭<𝓗>
1: ──X─────│───├G²─╭G²─├G²─┤ ├<𝓗>
2: ──X─╭G²─│───│───├G²─│───┤ ├<𝓗>
3: ──X─├G²─├G²─│───│───│───┤ ├<𝓗>
4: ────├G²─├G²─│───│───├G²─┤ ├<𝓗>
5: ────╰G²─│───│───├G²─╰G²─┤ ├<𝓗>
6: ────────│───├G²─╰G²─────┤ ├<𝓗>
7: ────────╰G²─╰G²─────────┤ ╰<𝓗>

n = 6,  E = -5.73249869 H, Largest Gradient = 0.118
0: ──X─────╭G²─╭G²─────╭G²─────────┤ ╭<𝓗>
1: ──X─────│───├G²─╭G²─├G²─╭G²─────┤ ├<𝓗>
2: ──X─╭G²─│───│───├G²─│───├G²─╭G²─┤ ├<𝓗>
3: ──X─├G²─├G²─│───│───│───│───├G²─┤ ├<

https://chem.libretexts.org/Bookshelves/Physical_and_Theoretical_Chemistry_Textbook_Maps/Supplemental_Modules_(Physical_and_Theoretical_Chemistry)/Quantum_Mechanics/10%3A_Multi-electron_Atoms/8%3A_The_Helium_Atom#:~:text=Therefore%20the%20ground%2Dstate%20energy,79.02%20eV%3D%E2%88%922.90372%20hartrees.

For helium ( Z=2
 ), this gives  −2.84765
  hartrees, an error of about  2%
   (E0=−2.90372)
 .

In [16]:
import pennylane as qml
H2Odatasets = qml.data.load("qchem", molname="NH3", basis="STO-3G")
print('fci_energy',H2Odatasets[0].fci_energy)
print('vqe_energy',H2Odatasets[0].vqe_energy)

print(H2Odatasets)

fci_energy -55.519197646545145
vqe_energy -55.51822494385617
[<Dataset = molname: NH3, basis: STO-3G, bondlength: 1.013, attributes: ['basis', 'basis_rot_groupings', ...]>]
